# **Modeling and Evaluation**

## Objectives

- Address Business Requirement 2: Develop a model to determine whether a given leaf is infected with powdery mildew.

- Implement machine learning techniques to train and evaluate a classification model.

## Inputs
- Dataset Directories:
- `inputs/mildew_dataset_dataset/cherry-leaves/train`
- `inputs/mildew_dataset_dataset/cherry-leaves/test`
- `inputs/mildew_dataset_dataset/cherry-leaves/validation`
- Image Shape Embeddings: Precomputed embeddings from the Data Visualization Notebook.

## Outputs

- Image distribution plot for training, validation, and test sets.

- Implementation of image augmentation techniques.

- Class indices mapping for label interpretation during inference.

- Trained machine learning model.

- Saved trained model for future inference.

- Learning curve plot illustrating model performance over epochs.

- Model evaluation metrics saved as a pickle file.

- Prediction on a randomly selected image.

## Additional Comments

- This notebook focuses on developing and training a classification model using the structured dataset.

- Performance evaluation ensures that the model meets the defined business requirement.

- Proper validation and testing procedures ensure model robustness before deployment.

- The trained model will serve as the backbone for the mildew detection application, aiding in real-time predictions.

- This version includes hyperparameter tuning for Merit Level performance.



---

## Set Up Environment

### Import Packages

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread

### Set Working Directory

In [ ]:
cwd = os.getcwd()
os.chdir('/workspace/powdery-mildew-detector')
print("You set a new current directory")

In [ ]:
work_dir = os.getcwd()
work_dir

### Set Input Directories

In [5]:
# Set train, validation and test paths
my_data_dir = 'inputs/mildew_dataset/cherry-leaves'
train_path = my_data_dir + '/train'
val_path = my_data_dir + '/validation'
test_path = my_data_dir + '/test'

### Set Output Directory

In [ ]:
version = 'v1'
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(work_dir) and version in os.listdir(work_dir + '/outputs'):
    print('Old version is already available create a new version.')
    pass
else:
    os.makedirs(name=file_path)

### Set Labels

In [ ]:
# Set the lables for the images
labels = os.listdir(train_path)

print(
    f"Project Labels: {labels}"
)

### Set Image Shape

In [ ]:
## Import saved image shape embedding
import joblib
version = 'v1'
image_shape = joblib.load(filename=f"outputs/{version}/image_shape.pkl")
image_shape

---

### Number of Images In train, Test and Validation Data

In [ ]:
import pandas as pd

# Create an empty dictionary to store data
data = {
    'Set': [],
    'Label': [],
    'Frequency': []
}

# List of dataset folders
folders = ['train', 'validation', 'test']

# Go through each folder and label to count the images
for folder in folders:
    for label in labels:
        row = {
            'Set': folder,
            'Label': label,
            'Frequency': int(len(os.listdir(my_data_dir + '/' + folder + '/' + label)))  
        }
        for key, value in row.items():
            data[key].append(value)
        print(
            f"* {folder} - {label}: {len(os.listdir(my_data_dir+'/'+ folder + '/' + label))} images")

# Convert the dictionary into a DataFrame
df_freq = pd.DataFrame(data)

print("\n")

# Set plot style
sns.set_style("whitegrid")
plt.figure(figsize=(8, 5))

# Create a bar chart to show image distribution
sns.barplot(data=df_freq, x='Set', y='Frequency', hue='Label')
plt.savefig(f'{file_path}/labels_distribution.png',
            bbox_inches='tight', dpi=150)
plt.show()

---

## Image Data Agmentation

Image Data Generator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Initialize image data generator

In [11]:
augmented_image_data = ImageDataGenerator(rotation_range=20,
                                          width_shift_range=0.10,
                                          height_shift_range=0.10,
                                          shear_range=0.1,
                                          zoom_range=0.1,
                                          horizontal_flip=True,
                                          vertical_flip=True,
                                          fill_mode='nearest',
                                          rescale=1./255
                                          )

Augment training image dataset

In [ ]:
batch_size = 20
train_set = augmented_image_data.flow_from_directory(
    train_path,
    target_size=image_shape[:2],
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
    )

train_set.class_indices

Augment validation image dataset

In [ ]:
validation_set = ImageDataGenerator(
    rescale=1./255).flow_from_directory(val_path,
                                        target_size=image_shape[:2],
                                        color_mode='rgb',
                                        batch_size=batch_size,
                                        class_mode='binary',
                                        shuffle=False
                                        )

validation_set.class_indices

Augment test image dataset

In [ ]:
test_set = ImageDataGenerator(
    rescale=1./255).flow_from_directory(test_path,
                                        target_size=image_shape[:2],
                                        color_mode='rgb',
                                        batch_size=batch_size,
                                        class_mode='binary',
                                        shuffle=False
                                        )

test_set.class_indices

### Plot Augmented Training Images

In [ ]:
for _ in range(3):
    img, label = next(train_set)
    print(img.shape)  # (1,256,256,3)
    plt.imshow(img[0])
    plt.show()

### Plot Augmented Validation Images

In [ ]:
for _ in range(3):
    img, label = next(validation_set)
    print(img.shape)  # (1,256,256,3)
    plt.imshow(img[0])
    plt.show()

### Plot Augmented Test Images

In [ ]:
for _ in range(3):
    img, label = next(test_set)
    print(img.shape)  # (1,256,256,3)
    plt.imshow(img[0])
    plt.show()

### Save Class Indices

In [ ]:
joblib.dump(value=train_set.class_indices,
            filename=f"{file_path}/class_indices.pkl")

---

## Model Creation

- Import model packages

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

- Model

In [ ]:
def create_tf_model(input_shape):
    """
    Create a CNN model.
    """
    model = Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),  # Dropout to prevent overfitting
        Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',  # Adam optimizer is commonly used for image classification
                  metrics=['accuracy', 'Precision', 'Recall', 'AUC'] # Added precision, recall, and AUC
                  )

    return model

Model Summary

In [ ]:
create_tf_model().summary()

Early Stopping 

In [32]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True) # Early stopping to prevent overfitting.  The model will stop training if the validation loss doesn't improve for 5 epochs.

### Hyperparameter Tuning

In [ ]:
best_accuracy = 0
best_hyperparams = {}
version = 'v2'
file_path = f'outputs/{version}'

for filters_1 in: # Example: Trying 2 values for the number of filters in the first Conv2D layer
    for filters_2 in: # Example: Trying 2 values for the number of filters in the second Conv2D layer
        for dropout_rate in: # Example: Trying 2 values for dropout rate
            print(f"Training with filters_1={filters_1}, filters_2={filters_2}, dropout_rate={dropout_rate}")
            model = create_tf_model(image_shape)
            model.layers.filters = filters_1
            model.layers.filters = filters_2
            model.layers.rate = dropout_rate

            history = model.fit(train_set,
                              epochs=10, # Reduced number of epochs for hyperparameter tuning
                              steps_per_epoch=len(train_set.classes) // batch_size,
                              validation_data=validation_set,
                              callbacks=[early_stop],
                              verbose=1)

            if history.history['val_accuracy'][-1] > best_accuracy:
                best_accuracy = history.history['val_accuracy'][-1]
                best_hyperparams = {'filters_1': filters_1, 'filters_2': filters_2, 'dropout_rate': dropout_rate}
                best_model = model

print(f"Best Hyperparameters: {best_hyperparams}")
print(f"Best Validation Accuracy: {best_accuracy}")


###  Fit model for model training (using best hyperparameters)

In [ ]:
model = best_model # Use the best model found during tuning.
model.fit(train_set,
          epochs=25, # Full training with best parameters
          steps_per_epoch=len(train_set.classes) // batch_size,
          validation_data=validation_set,
          callbacks=[early_stop],
          verbose=1
          )

### Save model

In [ ]:
model.save(f'{file_path}/mildew_detector_model.keras')

---

## Model Performance

### Model Learning Curve

In [ ]:
losses = pd.DataFrame(model.history.history)

sns.set_style("whitegrid")
losses[['loss', 'val_loss']].plot(style='.-')
plt.title("Loss")
plt.savefig(f'{file_path}/model_training_losses.png',
            bbox_inches='tight', dpi=150)
plt.show()

print("\n")
losses[['accuracy', 'val_accuracy']].plot(style='.-')
plt.title("Accuracy")
plt.savefig(f'{file_path}/model_training_acc.png',
            bbox_inches='tight', dpi=150)
plt.show()


### Interpretation of Learning Curves:
These plots show the training and validation loss and accuracy over epochs.  Ideally, we want to see the training and validation curves converging, indicating that the model is learning effectively.  If the training loss is much lower than the validation loss, it might suggest overfitting.  If both losses are high, it could indicate underfitting.

## Model Evaluation

Load saved model

In [ ]:
from keras.models import load_model
model = load_model('outputs/v1/mildew_detector_model.h5')

Evaluate model on test set

In [ ]:
evaluation = model.evaluate(test_set)

### Save Evaluation Pickle

In [ ]:
joblib.dump(value=evaluation,
            filename=f"outputs/v1/evaluation.pkl")

### Predict on New Data

Load a random image as PIL

In [ ]:
from tensorflow.keras.preprocessing import image

pointer = 50
label = labels[0]  

pil_image = image.load_img(test_path + '/' + label + '/' + os.listdir(test_path+'/' + label)[pointer],
                           target_size=image_shape, color_mode='rgb')
print(f'Image shape: {pil_image.size}, Image mode: {pil_image.mode}')
pil_image

Convert image to array and prepare for prediction

In [ ]:
my_image = image.img_to_array(pil_image)
my_image = np.expand_dims(my_image, axis=0)/255
print(my_image.shape)

Predict class probabilities

In [ ]:
pred_proba = model.predict(my_image)[0, 0]

target_map = {v: k for k, v in train_set.class_indices.items()}
pred_class = target_map[pred_proba > 0.5]

if pred_class == target_map[0]:
    pred_proba = 1 - pred_proba

print(pred_proba)
print(pred_class)

---

## Conclusions and Next Steps

### Conclusions
- The selected CNN model, after hyperparameter tuning, achieved [accuracy] on the test set.
- The model demonstrates good performance in classifying healthy and infected cherry leaves.
- The learning curves indicate [interpretation of the curves].
- The confusion matrix shows [interpretation of the matrix].

### Next Steps
- Further optimization of the model architecture and hyperparameters could potentially improve performance.
- Deploy the trained model as part of a Streamlit application for practical use.
- Explore the possibility of extending the model to other crops and diseases.